In [ ]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BenchmarkTools
import Markdown; Base.showable(::MIME"text/markdown", ::Markdown.MD) = false # to make sth like """?replace"""  works for vscode backend, remember to choose Jupyter kernel
include(pkg_folder*"src/initialize.jl")
Threads.nthreads() # check number of threads

  Activating project at `~/Realizibility_index/bnc_julia`


24

In [2]:
    # I  A  Bs As Bs2 E As2 B C1 C2 C3 C4
N = [1   1  0   0  0  0  0  0  -1  0  0  0
     0   0  1   1  0  0  0  0   0 -1  0  0
     0   0  0   0  1  1  0  0   0  0 -1  0
     0   0  0   0  0  0  1  1   0  0  0 -1]
model = Bnc(N=N)

Bnc{Int8}([1 1 … 0 0; 0 0 … 0 0; 0 0 … -1 0; 0 0 … 0 -1], [1 0 … 0 0; 0 1 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 1], 4, 12, 8, Num[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉, x₁₀, x₁₁, x₁₂], Num[q₁, q₂, q₃, q₄, q₅, q₆, q₇, q₈], Num[K₁, K₂, K₃, K₄], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], nothing, Vertex[], Bool[], false, Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.3010299956639812, 0.0, 0.0, 0.0, 0.0], false, sparse([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10, 11, 11, 12, 12], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 8, 12), sparse([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10, 11, 11, 12, 12], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 8, 12), 

In [3]:
find_all_vertices!(model)

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 256 vertices found and 256 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.


256-element Vector{Vector{Int8}}:
 [1, 2, 3, 4, 5, 6, 7, 8]
 [1, 2, 3, 4, 5, 6, 7, 12]
 [1, 2, 3, 4, 5, 6, 12, 8]
 [1, 2, 3, 4, 5, 6, 12, 12]
 [1, 2, 3, 4, 5, 11, 7, 8]
 [1, 2, 3, 4, 5, 11, 7, 12]
 [1, 2, 3, 4, 5, 11, 12, 8]
 [1, 2, 3, 4, 5, 11, 12, 12]
 [1, 2, 3, 4, 11, 6, 7, 8]
 [1, 2, 3, 4, 11, 6, 7, 12]
 [1, 2, 3, 4, 11, 6, 12, 8]
 [1, 2, 3, 4, 11, 6, 12, 12]
 [1, 2, 3, 4, 11, 11, 7, 8]
 ⋮
 [9, 9, 10, 10, 5, 11, 7, 8]
 [9, 9, 10, 10, 5, 11, 7, 12]
 [9, 9, 10, 10, 5, 11, 12, 8]
 [9, 9, 10, 10, 5, 11, 12, 12]
 [9, 9, 10, 10, 11, 6, 7, 8]
 [9, 9, 10, 10, 11, 6, 7, 12]
 [9, 9, 10, 10, 11, 6, 12, 8]
 [9, 9, 10, 10, 11, 6, 12, 12]
 [9, 9, 10, 10, 11, 11, 7, 8]
 [9, 9, 10, 10, 11, 11, 7, 12]
 [9, 9, 10, 10, 11, 11, 12, 8]
 [9, 9, 10, 10, 11, 11, 12, 12]

In [71]:
f(tE,tI,tA,tB,tAs,tBs,K1,K2,K3,K4)=begin
    f(Et, St, K, n=nothing) = begin
        if n==1 return K > Et && K > St
        elseif n==3 return St > Et && St > K
        elseif n==2 return Et > St && Et > K
        elseif isnothing(n) return (Et < K) || (St > Et)
        else error("n should be 1,2,3 or nothing")  
        end
    end
    judge1 =true #f(tI, tA, K1,3)
    judge2 = true #f(tBs, tAs, K2,1)
    judge3 = f(tAs,tB,K3,3)
    judge4= f(tE, tBs, K4,3)
    return judge1 && judge2 && judge3 && judge4
end



f (generic function with 1 method)

In [8]:
f(tE1,tS1,K1,tE2, tS2, K2)=begin
    f(Et, St, K, n=nothing) = begin
        if n==1 return K > Et && K > St
        elseif n==3 return St > Et && St > K
        elseif n==2 return Et > St && Et > K
        elseif isnothing(n) return (Et < K) || (St > Et)
        else error("n should be 1,2,3 or nothing")  
        end
    end
    judge3 = f(tE1,tS1,K1,3)
    judge4= f(tE2, tS2, K2,3)
    return judge3 && judge4
end

f (generic function with 1 method)

In [19]:
get_rand() =begin
    while true 
        (tE1,tS1,K1,tE2, tS2, K2) = randomize(6)
        # tS =log10(exp10(tS1) + exp10(tS2))
        tS = maximum([tS1,tS2])
        if tS > tE1 && tS > tE2
            return (tE1,tS1,K1,tE2, tS2, K2)
        end
    end
end

get_rand (generic function with 1 method)

In [20]:
N = 100_000_000
countmap([f(get_rand()...) for _ in 1:N])

Dict{Bool, Int64} with 2 entries:
  0 => 77783735
  1 => 22216265

In [17]:
function get_center_margin(count::Int, N::Int)
    z=1.96  # for 95% confidence interval
    if count == 0
        return 0.0, 0.0
    end
    P_hat = count / N
    denom = 1 + z^2 / N
    center = (P_hat + z^2 / (2 * N)) / denom
    margin = (z / denom) * sqrt(P_hat * (1 - P_hat) / N + z^2 / (4 * N^2))
    return center, margin
end

get_center_margin (generic function with 1 method)

In [18]:
# get_center_margin(15657829, 100_000_000)
# get_center_margin(6246057, 100_000_000)
# get_center_margin(317565, 100_000_000)
# get_center_margin(847082, 100_000_000)
# get_center_margin(846591, 100_000_000)
get_center_margin(2213259, 100_000_00)
# 847082

(0.2213260070554011, 0.00025730605707254)

In [15]:
2/9

0.2222222222222222

In [53]:
(1/2)^4

0.0625

In [64]:
(0.00317+0.00847+0.00847)

0.020110000000000003